# Traffic Sign Classification with Keras



In [1]:
from urllib.request import urlretrieve
from os.path import isfile
from tqdm import tqdm

class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

if not isfile('train.p'):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc='Train Dataset') as pbar:
        urlretrieve(
            'https://s3.amazonaws.com/udacity-sdc/datasets/german_traffic_sign_benchmark/train.p',
            'train.p',
            pbar.hook)

if not isfile('test.p'):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc='Test Dataset') as pbar:
        urlretrieve(
            'https://s3.amazonaws.com/udacity-sdc/datasets/german_traffic_sign_benchmark/test.p',
            'test.p',
            pbar.hook)

print('Training and Test data downloaded.')

Train Dataset: 120MB [00:06, 17.2MB/s]                           
Test Dataset: 38.8MB [00:01, 30.4MB/s]                           

Training and Test data downloaded.


## Overview

Here are the steps you'll take to build the network:

1. Load the training data.
2. Preprocess the data.
3. Build a feedforward neural network to classify traffic signs.
4. Build a convolutional neural network to classify traffic signs.
5. Evaluate the final neural network on testing data.


In [0]:
import pickle
import numpy as np
import math

## Load the Data

Start by importing the data from the pickle file.

In [16]:
with open('train.p', 'rb') as f:
    data = pickle.load(f)

type(data)

dict

In [17]:
print(data.keys())



dict_keys(['features', 'labels'])


In [28]:
# TODO: Load the feature data to the variable X_train
X_train = data['features']
# TODO: Load the label data to the variable y_train
y_train = data['labels']

print(X_train.shape)
print(y_train.shape)
print(y_train[-1])
type(X_train)

(39209, 32, 32, 3)
(39209,)
42


numpy.ndarray

## Preprocess the Data

1. Shuffle the data
2. Normalize the features using Min-Max scaling between -0.5 and 0.5
3. One-Hot Encode the labels

### Shuffle the data
Hint: You can use the [scikit-learn shuffle](http://scikit-learn.org/stable/modules/generated/sklearn.utils.shuffle.html) function to shuffle the data.

In [0]:
# TODO: Shuffle the data
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)

In [32]:
print(y_train[0]) # label for the 1st image
print(y_train[-1])  # label for the last image

5
1


### Normalize the features


In [0]:
# TODO: Normalize the data features to the variable X_normalized
X_min = 0
X_max = 255
a = -0.5
b = 0.5
X_normalized = a + ((X_train - X_min)*(b-a)/ (X_max - X_min))


In [34]:
print(X_normalized[0])

[[[-0.40980392 -0.4254902  -0.42941176]
  [-0.39803922 -0.41372549 -0.40980392]
  [-0.37843137 -0.39411765 -0.39803922]
  ...
  [-0.34313725 -0.37843137 -0.39019608]
  [-0.34313725 -0.37058824 -0.39019608]
  [-0.33921569 -0.3627451  -0.38627451]]

 [[-0.40980392 -0.4254902  -0.42941176]
  [-0.40196078 -0.41372549 -0.40588235]
  [-0.38627451 -0.39803922 -0.39803922]
  ...
  [-0.34313725 -0.3745098  -0.38627451]
  [-0.33921569 -0.36666667 -0.38627451]
  [-0.32745098 -0.35490196 -0.38235294]]

 [[-0.40588235 -0.4254902  -0.42941176]
  [-0.40980392 -0.42156863 -0.41764706]
  [-0.39019608 -0.39803922 -0.40196078]
  ...
  [-0.33921569 -0.37058824 -0.38627451]
  [-0.32745098 -0.35490196 -0.3745098 ]
  [-0.30392157 -0.33921569 -0.36666667]]

 ...

 [[-0.32352941 -0.37058824 -0.39803922]
  [-0.34313725 -0.3745098  -0.39803922]
  [-0.35882353 -0.3745098  -0.39411765]
  ...
  [-0.30392157 -0.33921569 -0.3627451 ]
  [-0.3        -0.33921569 -0.36666667]
  [-0.30392157 -0.34313725 -0.37843137]]

 [

### One-Hot Encode the labels
Hint: You can use the [scikit-learn LabelBinarizer](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelBinarizer.html) function to one-hot encode the labels.

In [0]:
# TODO: One Hot encode the labels to the variable y_one_hot
from sklearn import preprocessing 
lb = preprocessing.LabelBinarizer()
lb.fit(y_train)
y_one_hot =lb.transform(y_train)

In [38]:
y_one_hot[-1]

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

## Keras Sequential Model
```python
from keras.models import Sequential

# Create the Sequential model
model = Sequential()
```
The `keras.models.Sequential` class is a wrapper for the neural network model. Just like many of the class models in scikit-learn, it provides common functions like `fit()`, `evaluate()`, and `compile()`.  We'll cover these functions as we get to them.  Let's start looking at the layers of the model.

## Keras Layer
A Keras layer is just like a neural network layer.  It can be fully connected, max pool, activation, etc.  You can add a layer to the model using the model's `add()` function.  For example, a simple model would look like this:
```python
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten

# Create the Sequential model
model = Sequential()

# 1st Layer - Add a flatten layer
model.add(Flatten(input_shape=(32, 32, 3)))

# 2nd Layer - Add a fully connected layer
model.add(Dense(100))

# 3rd Layer - Add a ReLU activation layer
model.add(Activation('relu'))

# 4th Layer - Add a fully connected layer
model.add(Dense(60))

# 5th Layer - Add a ReLU activation layer
model.add(Activation('relu'))
```
Keras will automatically infer the shape of all layers after the first layer.  This means you only have to set the input dimensions for the first layer.

The first layer from above, `model.add(Flatten(input_shape=(32, 32, 3)))`, sets the input dimension to (32, 32, 3) and output dimension to (3072=32\*32\*3).  The second layer takes in the output of the first layer and sets the output dimenions to (100).  This chain of passing output to the next layer continues until the last layer, which is the output of the model.

## Build a Multi-Layer Feedforward Network

Build a multi-layer feedforward neural network to classify the traffic sign images.

1. Set the first layer to a `Flatten` layer with the `input_shape` set to (32, 32, 3)
2. Set the second layer to `Dense` layer width to 128 output. 
3. Use a ReLU activation function after the second layer.
4. Set the output layer width to 43, since there are 43 classes in the dataset.
5. Use a softmax activation function after the output layer.

To get started, review the Keras documentation about [models](https://keras.io/models/sequential/) and [layers](https://keras.io/layers/core/).

The Keras example of a [Multi-Layer Perceptron](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py) network is similar to what you need to do here. Use that as a guide, but keep in mind that there are a number of differences.

In [0]:
# TODO: Build a Multi-layer feedforward neural network with Keras here.
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten 
model = Sequential()
model.add(Flatten(input_shape = (32,32,3)))  # This is the input layer
model.add(Dense(128))         # This is the hidden layer with 128 neurons
model.add(Activation('relu'))
model.add(Dense(43))         # This is the output layer
model.add(Activation('softmax'))

,## Training a Sequential Model
You built a multi-layer neural network in Keras, now let's look at training a neural network.
```python
from keras.models import Sequential
from keras.layers.core import Dense, Activation

model = Sequential()
...

# Configures the learning process and metrics
model.compile('sgd', 'mean_squared_error', ['accuracy'])

# Train the model
# History is a record of training loss and metrics
history = model.fit(x_train_data, Y_train_data, batch_size=128, nb_epoch=2, validation_split=0.2)

# Calculate test score
test_score = model.evaluate(x_test_data, Y_test_data)
```
The code above configures, trains, and tests the model.  The line `model.compile('sgd', 'mean_squared_error', ['accuracy'])` configures the model's optimizer to `'sgd'`(stochastic gradient descent), the loss to `'mean_squared_error'`, and the metric to `'accuracy'`.  

You can find more optimizers [here](https://keras.io/optimizers/), loss functions [here](https://keras.io/objectives/#available-objectives), and more metrics [here](https://keras.io/metrics/#available-metrics).

To train the model, use the `fit()` function as shown in `model.fit(x_train_data, Y_train_data, batch_size=128, nb_epoch=2, validation_split=0.2)`.  The `validation_split` parameter will split a percentage of the training dataset to be used to validate the model.  The model can be further tested with the test dataset using the `evaluate()` function as shown in the last line.

## Train the Network

1. Compile the network using adam optimizer and categorical_crossentropy loss function.
2. Train the network for ten epochs and validate with 20% of the training data.

In [40]:
# TODO: Compile and train the model here.

model.compile('adam', 'categorical_crossentropy', ['accuracy'])
history = model.fit(X_normalized, y_one_hot, batch_size=120, nb_epoch=10, validation_split= 0.2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until


Train on 31367 samples, validate on 7842 samples
Epoch 1/10
31367/31367 [==============================] - 5s 168us/step - loss: 1.7498 - acc: 0.5430 - val_loss: 1.0329 - val_acc: 0.7313
Epoch 2/10
31367/31367 [==============================] - 5s 145us/step - loss: 0.8184 - acc: 0.7877 - val_loss: 0.7510 - val_acc: 0.7830
Epoch 3/10
31367/31367 [==============================] - 4s 129us/step - loss: 0.5884 - acc: 0.8483 - val_loss: 0.5309 - val_acc: 0.8628
Epoch 4/10
31367/31367 [==============================] - 4s 129us/step - loss: 0.4665 - acc: 0.8800 - val_loss: 0.4400 - val_acc: 0.8777
Epoch 5/10
31367/31367 [==============================] - 4s 130us/step - loss: 0.3907 - acc: 0.8968 - val_loss: 0.4481 - val_acc: 0.8676
Epoch 6/10
31367/31367 [==============================] - 4s 131us/step - loss: 0.3509 - acc: 0.9078 - val_loss: 0.3376 - val_acc: 0.9208
Epoch 7/10
31367/31367 [==============================] - 4s 131us/step - loss: 0.3004 - acc: 0.9214 - val_loss: 0.4242 - v

## Convolutions
1. Re-construct the previous network
2. Add a [convolutional layer](https://keras.io/layers/convolutional/#convolution2d) with 32 filters, a 3x3 kernel, and valid padding before the flatten layer.
3. Add a ReLU activation after the convolutional layer.

Hint 1: The Keras example of a [convolutional neural network](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py) for MNIST would be a good example to review.

In [41]:
# TODO: Re-construct the network and add a convolutional layer before the flatten layer.
from keras.models import Sequential
from keras.layers.core import Dense, Flatten, Activation
from keras.layers import Convolution2D
model = Sequential()
model.add(Convolution2D(32, 3, 3, border_mode='valid', input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(Flatten())  # the convoluted image if flattened to a single vector
model.add(Dense(128))  # The flattened image is fed to the hidden layer (fully-connected layer)
model.add(Activation('relu'))
model.add(Dense(43))   # final output layer
model.add(Activation('softmax'))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(32, 32, 3..., padding="valid")`
  """


## Pooling
1. Re-construct the network
2. Add a 2x2 [max pooling layer](https://keras.io/layers/pooling/#maxpooling2d) immediately following your convolutional layer.

In [42]:
# TODO: Re-construct the network and add a pooling layer after the convolutional layer.
from keras.models import Sequential
from keras.layers.core import Dense, Flatten, Activation
from keras.layers import Convolution2D, pooling
model = Sequential()
model.add(Convolution2D(32, 3, 3, border_mode='valid', input_shape=(32,32,3)))  
# output convulted image has size (30,30,32)
model.add(pooling.MaxPooling2D(pool_size=(2, 2))) 
model.add(Activation('relu'))
model.add(Flatten()) # is to convert the outout of the MaxPoling LAyer into a single row of pixels >> 1-D
model.add(Dense(128))  # H.L. with 128 neurons >> fully-connected layer
model.add(Activation('relu'))
model.add(Dense(43))   # output layer with 43 neurons are there as 43 classes
model.add(Activation('softmax'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(32, 32, 3..., padding="valid")`
  """


## Dropout
1. Re-construct the network
2. Add a [dropout](https://keras.io/layers/core/#dropout) layer after the pooling layer. Set the dropout rate to 50%.

In [43]:
# TODO: Re-construct the network and add dropout after the pooling layer.
from keras.models import Sequential
from keras.layers.core import Dense, Flatten, Activation, Dropout
from keras.layers import Convolution2D, pooling
model = Sequential()
model.add(Convolution2D(32, 3, 3, border_mode='valid', input_shape=(32,32,3)))
model.add(pooling.MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Flatten()) 
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(43))
model.add(Activation('softmax'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(32, 32, 3..., padding="valid")`
  """


## Optimization
Congratulations! You've built a neural network with convolutions, pooling, dropout, and fully-connected layers, all in just a few lines of code.

Have fun with the model and see how well you can do! Add more layers, or regularization, or different padding, or batches, or more training epochs.

What is the best validation accuracy you can achieve?

In [44]:
# TODO: Build a model
from keras.models import Sequential
from keras.layers.core import Dense, Flatten, Activation, Dropout
from keras.layers import Convolution2D, pooling
model = Sequential()
model.add(Convolution2D(32, 3, 3, border_mode='valid', input_shape=(32,32,3)))
model.add(pooling.MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Flatten()) 
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(43))
model.add(Activation('softmax'))

# TODO: Compile and train the model
model.compile('adam', 'categorical_crossentropy', ['accuracy'])
history = model.fit(X_normalized, y_one_hot, batch_size=128, nb_epoch=10, validation_split=0.2)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(32, 32, 3..., padding="valid")`
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 31367 samples, validate on 7842 samples
Epoch 1/10
31367/31367 [==============================] - 40s 1ms/step - loss: 1.6369 - acc: 0.5692 - val_loss: 0.7254 - val_acc: 0.7979
Epoch 2/10
31367/31367 [==============================] - 40s 1ms/step - loss: 0.5086 - acc: 0.8677 - val_loss: 0.3277 - val_acc: 0.9263
Epoch 3/10
31367/31367 [==============================] - 40s 1ms/step - loss: 0.3077 - acc: 0.9211 - val_loss: 0.2222 - val_acc: 0.9473
Epoch 4/10
31367/31367 [==============================] - 39s 1ms/step - loss: 0.2237 - acc: 0.9421 - val_loss: 0.1730 - val_acc: 0.9612
Epoch 5/10
31367/31367 [==============================] - 39s 1ms/step - loss: 0.1791 - acc: 0.9538 - val_loss: 0.1444 - val_acc: 0.9672
Epoch 6/10
31367/31367 [==============================] - 39s 1ms/step - loss: 0.1525 - acc: 0.9592 - val_loss: 0.1361 - val_acc: 0.9685
Epoch 7/10
31367/31367 [==============================] - 39s 1ms/step - loss: 0.1344 - acc: 0.9643 - val_loss: 0.1235 - val_acc:

**Best Validation Accuracy:** 98.07%

## Testing
Once you've picked out your best model, it's time to test it.

Load up the test data and use the [`evaluate()` method](https://keras.io/models/model/#evaluate) to see how well it does.

Hint 1: The `evaluate()` method should return an array of numbers. Use the [`metrics_names`](https://keras.io/models/model/) property to get the labels.

In [45]:
# TODO: Load test data
with open('test.p', 'rb') as f:
    data = pickle.load(f)
X_test = data['features']
y_test = data['labels']

# TODO: Normalize the data features to the variable X_normalized
X_min = 0
X_max = 255
a = -0.5
b = 0.5
X_test_normalized = a + ((X_test - X_min)*(b-a)/ (X_max - X_min))

# TODO: Preprocess data & one-hot encode the labels
from sklearn import preprocessing 
lb = preprocessing.LabelBinarizer()
lb.fit(y_test)
y_test_one_hot =lb.transform(y_test)

# TODO: Evaluate model on test data
metrics = model.evaluate(X_test_normalized, y_test_one_hot)
for metric_i in range(len(model.metrics_names)):
    metric_name = model.metrics_names[metric_i]
    metric_value = metrics[metric_i]
    print('{}: {}'.format(metric_name, metric_value))

12630/12630 [==============================] - 6s 483us/step
loss: 0.4715543721983193
acc: 0.8998416468630871
